# Old or not

## Business Understanding: 
Van volwassen beroemdheden is het vaak lastig te zeggen hoe oud ze zijn. Nu gaan we kijken of we een neuraal netwerk kunnen bouwen wat dit beter kan schatten dan wij. We nemens als grens de compleet arbitraire waarde van 42 jaar. Iedereen tussen de 21 en 42 noemen we 'not old' en iedereen boven deze grens definieren we als 'old' 

We gebruiken hiervoor de IMDB faces database met daarin meer dan 460.000 gezichten van een dikke 20.000 beroemdheden afkomstig van IMDB (zie https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/ voor uitleg en details) en hierbij de IMDB informatie over o.a. leeftijd en geslacht. We downloaden hier de de cropped IMDB dataset( https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar). De metadata wordt in .mat (matlab) bestand meegeleverd. 

- We beperken ons tot volwassen beroemdheden. <u>Iedereen onder de 21 jaar mag je overslaan.</u>
- Er zitten nogal wat foto's met meerdere gezichten of geen gezicht op. <u>Deze mag je eruit filteren</u>
- We verwachten niet alleen de code maar (in het notebook) ook een <u>uitleg</u> van waarom je dingen hebt gedaan. 
- I.v.m. performance issues gebruiken we na opschoning alleen de <u>1e 50.000</u> records als data
- Gebruik <u>20% van je data als testset, gebruik 42 als random state</u>
- Eindig met de mogelijkheid om een jpg van bijv. jezelf in te lezen en d.m.v. het getrainde netwerk te bepalen of diegene wel of niet oud is.

Vul je antwoorden steeds op de juiste plaats in. Maak indien nodig extra cellen aan.

## Data Understanding: 

In [1]:
# Tips:
# - pickle voor tussentijds opslaan

In [30]:
#importeer de juiste packages
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

from sklearn.model_selection import train_test_split

from scipy.io import loadmat

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import sys
import os

sys.OpenCV_LOADER_DEBUG = True
import cv2


2024-09-19 18:17:22.946502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 18:17:23.020838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 18:17:23.042532: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 18:17:23.107189: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-19 18:17:24.713636: W tensorflow/compiler/tf2

In [3]:
# Constantes
DF_FILTERED_NAME = "cache_df_filtered.feather"  # https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d
COUNT_OF_FACES_TO_TRAIN_AND_TEST_ON = 50000

# afbeeldingen
WIDTH = HEIGHT = 100
SIZE = (WIDTH, HEIGHT)


In [4]:
#Lees de metadata in

mat = loadmat("imdb_crop/imdb.mat", squeeze_me=True)

In [ ]:
# bestudeer deze en maak voor iedere relevante variabele een aparte list aan
mat: dict = mat
imdb: np.ndarray = mat["imdb"]

(dob, photo_taken, full_path, gender, name, face_location, face_score, second_face_score, celeb_names, celeb_id) = imdb.tolist()

dob1 = dob[0]
print(dob1, type(dob1))

def convertDate(date):
    EPOCH = 719529
    return pd.to_datetime(int(date) - EPOCH, unit='D')

## Data preparation: 

In [6]:
#Bepaal op basis van de metadata hoe oud de personen op de foto's waren toen deze genomen werden (-1 als onbekend)

age_on_photo = []
# invalids = 0
for i in range(0, len(dob)):
    try:
        date = convertDate(dob[i])
        ageThen = photo_taken[i] - date.year
        age_on_photo.append(ageThen)
    except Exception as e:
        # invalids += 1
        age_on_photo.append(-1)

# print(invalids, "zijn/is ongeldig, dit is", (float(invalids)/len(dob)*100), "procent")

In [7]:
#voeg de ingelezen metadata en de berekende leeftijd samen tot een nieuwe dataset

df = pd.DataFrame(data={
    'name': name,
    'full_path': full_path,
    'age_on_photo': age_on_photo,
    'face_score': face_score,
    'second_face_score': second_face_score,
})

In [ ]:
#schoon de dataset op op basis van de bij de business understanding gegeven criteria.

OLDNESS_THRESHOLD = 42

## Filter: check of mensen minsten 21 jaar oud zijn
df_filtered = df[df['age_on_photo'] >= 21]

## Filter: alleen foto's waar gezichten op gevonden zijn
#       face_score: detector score (the higher the better).
#                   Inf implies that no face was found in the image and the face_location then just returns the entire image
df_filtered = df_filtered[df_filtered['face_score'] != np.Infinity]
df_filtered = df_filtered[df_filtered['face_score'] != -np.Infinity]

## Filter: alleen foto's waar maar een gezicht op staat
##      second_face_score: detector score of the face with the second highest score.
#                          This is useful to ignore images with more than one face.
#                          second_face_score is NaN if no second face was detected.
df_filtered = df_filtered[pd.isna(df_filtered['second_face_score'])]

df_filtered['oldness'] = df_filtered.apply(lambda row: int(row.age_on_photo >= OLDNESS_THRESHOLD), axis=1)

## Filter: max 50000
df_filtered = df_filtered.head(COUNT_OF_FACES_TO_TRAIN_AND_TEST_ON)

print(f"Gemiddelde oudheid: {df_filtered['oldness'].mean()}")

df_filtered.info()
df_filtered.head()

df_filtered.to_feather(DF_FILTERED_NAME)

![img](imdb_crop/01/nm0000001_rm946909184_1899-5-10_1968.jpg)

In [9]:
def readImage(path: str):
    cachePath = path.replace("jpg", "cached.jpg")
    if os.path.exists(cachePath):
        return cv2.imread(cachePath)

    img = cv2.imread(path)
    # Cursed conversion van BGR naar RGB
    img = img[...,::-1]

    img = cv2.resize(img, SIZE)

    # img = img / 255.0
    cv2.imwrite(cachePath, img)
    return img

images = [readImage("imdb_crop/" + path) for path in df_filtered['full_path']]


In [ ]:
# Lees de afbeeldingen in

plt.figure(figsize=(8,8))
for i in range(25):
    plt.subplot(5,5,i+1)  #vul het i-e image
    plt.xticks([]) #geen schaal bij x-as
    plt.yticks([]) #geen schaal bij y-as
    plt.imshow(images[i]) #orginele plaatjes nodig want de rest zit in een genormaliseerde vector
    plt.xlabel(df_filtered.iloc[i]['age_on_photo'])
plt.show()

In [ ]:
widths = np.array([img.shape[0] for img in images])
heights = np.array([img.shape[1] for img in images])

print(f"Width van {widths.min()} naar {widths.max()}")
print(f"Height van {heights.min()} naar {heights.max()}")

## Modeling: 

In [12]:

images_resized = np.array([cv2.resize(img, SIZE) for img in images])

In [13]:
images_resized_scaled = images_resized
# images_reshaped = images_resized_scaled.reshape((COUNT, WIDTH * HEIGHT * 3))
# images_reshaped.shape

In [ ]:
ages = np.array(df_filtered['oldness'])
ages.shape

age_on_photos = np.array(df_filtered['age_on_photo'])
age_on_photos.shape

names = np.array(df_filtered['name'])
names.shape

In [ ]:
indices = np.arange(0, COUNT_OF_FACES_TO_TRAIN_AND_TEST_ON)
train_images, test_images, train_ages, test_ages, train_age_on_photos, test_age_on_photos, train_names, test_names, train_indices, test_indices  = train_test_split(images_resized_scaled, ages, age_on_photos, names, indices, test_size=0.2, random_state=42)
ages

In [ ]:
# en valideer dit op de testdata

plt.figure(figsize=(8,8))
for i in range(min(25, len(train_images))):
    plt.subplot(5,5,i+1)  #vul het i-e image
    plt.xticks([]) #geen schaal bij x-as
    plt.yticks([]) #geen schaal bij y-as
    plt.imshow(train_images[i]) #orginele plaatjes nodig want de rest zit in een genormaliseerde vector
    plt.xlabel("Oud" if train_ages.tolist()[i] == 1 else "Jong")
    print(train_images[i].shape)
plt.show()

In [ ]:
# train het model
model = models.Sequential([
    layers.Dense(64, activation='relu', kernel_regularizer="l1"),
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(WIDTH, HEIGHT, 3), padding="SAME"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((3, 3)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((3, 3)),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax'),
])


model.compile(optimizer="ADAM",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

train_datagen = ImageDataGenerator(
    # rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    # vertical_flip=True,
)

data_flow = train_datagen.flow(train_images, tf.reshape(train_ages, (-1)), shuffle=True)
history = model.fit(data_flow, epochs=50, validation_data=(test_images, test_ages))

In [ ]:
model.summary()

In [18]:
# en valideer dit op de testdata


## Evaluation: 

In [ ]:
#plot de accuracy en validated accuracy
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
epochs=range(1, len(acc)+1)
plt.plot(epochs, acc, label='accuracy')
plt.plot(epochs, val_acc, label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
#toon de confusion matrix,
Y_predict= []
predict=model.predict([test_images]) #2d array
for i in range (len(predict)):
    Y_predict.append(predict[i].argmax()) #omzetten zodat ik alleen de max waarde krijg
cm=confusion_matrix(test_ages, Y_predict)
#print(cm) #lelijk maar functioneel

sns.heatmap(cm, annot=True, fmt='d', xticklabels=["Jong", "Oud"], yticklabels=["Jong", "Oud"])
# fmt='g' om te voorkomen dat de wetenschappelijke notatie (3.7e+02) gebruikt.

![Img](imdb_crop/93/nm0000093_rm3278212608_1963-12-18_2009.jpg)

In [ ]:
# en toon de een aantal afbeeldingen inclusief of ze volgens het model wel of niet oud zijn,
# doe dit laatste ook voor een paar eigen afbeeldingen


predict = model.predict(test_images[0:25])

predict.shape
plt.figure(figsize = (16, 16))
for i in range(25):
    ax = plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(test_images[i])

    prediction = round(predict[i].argmax())
    actual = round(test_ages[i])

    label = "Oud" if prediction == 1 else "Jong"
    label += f' ({test_age_on_photos[i]})'

    ax.xaxis.label.set_color('green' if prediction == actual else 'red')

    plt.xlabel(label)

    # ax = plt.subplot(10, 10, i * 2 + 2)
    # plt.xticks([])
    # plt.yticks([])
    # print(test_indices[i])
    # plt.xlabel(test_names[i])
plt.show()

Kunnen we met dit model betrouwbaar schatten of een beroemdheid oud is?
(type je antwoord en uitleg hieronder)

In [ ]:
tristan = readImage('1704671510929.jpeg')
tristan = cv2.resize(tristan, SIZE)
tristan = np.array([tristan])
tristan = tristan / 255.0
# tristan = tristan.reshape(WIDTH * HEIGHT * 3)
tristan.shape

In [ ]:
predict = model.predict([tristan])
predict: np.ndarray = predict[0]
predict = predict.argmax()
schatting = "oud" if predict == 1 else "jong"
print(f"Ik ben {schatting}")

In [ ]:
natalie = readImage('natalie.jpg')
natalie = cv2.resize(natalie, SIZE)
natalie = np.array([natalie])
natalie = natalie / 255.0
# tristan = tristan.reshape(WIDTH * HEIGHT * 3)
natalie.shape

predict = model.predict([natalie])
predict: np.ndarray = predict[0]
predict = predict.argmax()
schatting = "oud" if predict == 1 else "jong"
print(f"Natalie is {schatting}")